# Before Building Model

In [ ]:
from IPython.display import clear_output
!pip3 install flaml 
clear_output()

import pandas as pd
import numpy as np
import os
import random
import warnings
warnings.simplefilter('ignore')

from flaml import AutoML

TRAIN_PATH = "../input/tabular-playground-series-feb-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-feb-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-feb-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "row_id"
TARGET = "target"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

MODEL_TIME_BUDGET = 60*60
MODEL_METRIC = 'accuracy'
MODEL_TASK = "classification"
MODEL_LIST = ['lgbm', 'catboost', 'xgboost']
MODEL_LOG_FILE_PATH = "flaml_log.log"

CROSS_VAL_CV = 3


In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

train = train.drop([ID],axis=1)
test = test.drop([ID],axis=1)

def toLabelEncode(TRAIN_PATH,TEST_PATH):
    train = pd.read_csv(TRAIN_PATH)
    test = pd.read_csv(TEST_PATH)

    cat_col = train.describe(include="O").columns.tolist()
    
    for col in cat_col:
        if col == TARGET: continue
        
        train[col] = train[col].astype("category").cat.codes
        test[col] = test[col].astype("category").cat.codes
        
    return train,test

train,test = toLabelEncode(TRAIN_PATH,TEST_PATH)

def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

# Build Model

In [ ]:
X = train.drop([ID,TARGET],axis=1)
y = train[TARGET]

model = AutoML()
params = {
    "time_budget": MODEL_TIME_BUDGET,  
    "metric": MODEL_METRIC,
#     "estimator_list": MODEL_LIST, 
    "task": MODEL_TASK,
    "seed":SEED,
    "log_file_name":MODEL_LOG_FILE_PATH,
}
model.fit(X, y, **params)

In [ ]:
model.model.model

In [ ]:
from sklearn.model_selection import cross_validate
cross_validate(model.model.model, X, y, cv=CROSS_VAL_CV)

# After Building Model

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = model.predict(test)
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()